In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [9]:
input = 'complaints.csv'#sys.argv[1]
output_folder = '/output.txt'#sys.argv[2]

In [4]:
cpl = sc.textFile(input, use_unicode=True).cache()

In [31]:
def mapper(partitionId, records):
    if partitionId == 0:
        next(records)
    import csv
    reader = csv.reader(records,delimiter=',',quotechar='"')
    for r in reader:
        if len(r)>7 and type(r[0])==str and len(r[0])==10 and r[0][0]==('2'or'1'):
            yield r[0][0:4], \
                    r[1].lower().replace('"', "'") if ',' in r[1] else r[1].lower(), \
                    r[7].lower()

In [32]:
cpl.mapPartitionsWithIndex(mapper) \
    .map(lambda x: ((x[1],x[0],x[2]),1)) \
    .reduceByKey(lambda x,y: x+y) \
    .map(lambda x: ((x[0][0],x[0][1]),(x[1],1,x[1]))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], max(x[2],y[2]))) \
    .mapValues(lambda x: (x[0],x[1], round(x[2]/x[0]*100))) \
    .sortByKey() \
    .collect()
    #.saveAsTextFile(output_folder)

[(('bank account or service', '2012'), (12212, 98, 19)),
 (('bank account or service', '2013'), (13388, 164, 18)),
 (('bank account or service', '2014'), (14662, 258, 17)),
 (('bank account or service', '2015'), (17140, 215, 17)),
 (('bank account or service', '2016'), (21848, 230, 15)),
 (('bank account or service', '2017'), (6956, 174, 16)),
 (('checking or savings account', '2017'), (12763, 183, 17)),
 (('checking or savings account', '2018'), (21211, 214, 16)),
 (('checking or savings account', '2019'), (21740, 253, 15)),
 (('checking or savings account', '2020'), (4743, 166, 16)),
 (('consumer loan', '2012'), (1986, 84, 19)),
 (('consumer loan', '2013'), (3117, 159, 12)),
 (('consumer loan', '2014'), (5457, 358, 8)),
 (('consumer loan', '2015'), (7886, 599, 9)),
 (('consumer loan', '2016'), (9601, 665, 7)),
 (('consumer loan', '2017'), (3557, 425, 8)),
 (('credit card', '2011'), (1260, 33, 19)),
 (('credit card', '2012'), (15353, 76, 20)),
 (('credit card', '2013'), (13105, 108, 1